<hr>

<h2 style = "font-family:times; text-align: center;"> Creating a database with Google's Geodata </h2>

<hr>

<div style = "border: 2px solid black; padding: 10px; color: black; background-color: white;text-align: center; vertical-align:middle; font-family:times;">
    <h4> Creating an application to upload geodata from google's API and dysplay the information </h4>
</div>

<hr>

In [1]:
import sqlite3
import json
import codecs
import urllib.request, urllib.parse, urllib.error
import http
import time
import ssl
import sys

In [11]:
class geoload():
    
    def __init__(self):
        self._api_key = False
        
        self._conn = sqlite3.connect('geodata.sqlite')
        self._cur = self._conn.cursor()
        
        # Ignore SSL certificate errors
        self._ctx = ssl.create_default_context()
        self._ctx.check_hostname = False
        self._ctx.verify_mode = ssl.CERT_NONE

    def makeURL(self):
        if self._api_key is False:
            self._api_key = 42
            
            self._serviceurl = "http://py4e-data.dr-chuck.net/json?"
            
        else :
            self._serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

    def interactAPI(self):
        self._cur.execute('''
                CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)
                          ''')

        fh = open("where.data")
        
        count = 0
        
        for line in fh:
            if count > 200 :
                print('Retrieved 200 locations, restart to retrieve more')
                
                break

            address = line.strip()

            print('')

            self._cur.execute('''
                    SELECT geodata FROM Locations WHERE address= ?
                              ''', (memoryview(address.encode()), ))

            try:
                data = self._cur.fetchone()[0]

                print("\nFound in database: ",address)

                continue

            except:
                pass

            parms = dict()
            parms["address"] = address
            
            if self._api_key is not False: parms['key'] = self._api_key
            
            url = self._serviceurl + urllib.parse.urlencode(parms)

            print('\n...Retrieving...', url)
            
            uh = urllib.request.urlopen(url, context = self._ctx) 
            data = uh.read().decode()
            
            print('\nRetrieved ', len(data), ' characters', data[:20].replace('\n', ' '))
            
            count = count + 1

            try:
                js = json.loads(data)
                
            except:
                print(data) # We print in case unicode causes an error
                
                continue

            if ('status' not in js) or ((js['status'] != 'OK') and (js['status'] != 'ZERO_RESULTS')):
                print('\n==================== Failure To Retrieve ====================/n')                
                print("Data: \n", data)
                
                break

            self._cur.execute('''
                    INSERT INTO Locations (address, geodata)
                    VALUES ( ?, ? )
                              ''', (memoryview(address.encode()), memoryview(data.encode())))
            
            self._conn.commit()
            
            if (count % 10 == 0):
                print('\n...Pausing for a bit...')
                
                time.sleep(5)

        print("Run geodump.py to read the data from the database so you can vizualize it on a map.")

In [15]:
class geodump():
    
    def __init__(self):
        self._conn = sqlite3.connect('geodata.sqlite')
        
        self._cur = self._conn.cursor()

    def writeResults(self):
        self._cur.execute('''
                SELECT * FROM Locations
                          ''')
        
        fhand = codecs.open('where.js', 'w', "utf-8")
        
        fhand.write("myData = [\n")
        
        count = 0
        
        for row in self._cur:
            data = str(row[1].decode())
            
            try: js = json.loads(str(data))
            
            except: continue

            if not(('status' in js) and (js['status'] == 'OK')) : continue

            lat = js["results"][0]["geometry"]["location"]["lat"]
            lng = js["results"][0]["geometry"]["location"]["lng"]
            
            if lat == 0 or lng == 0 : continue
            
            where = js['results'][0]['formatted_address']            
            where = where.replace("'", "")
            
            try :
                print(where, lat, lng)

                count = count + 1
                
                if count > 1 : fhand.write(",\n")
                
                output = "[" + str(lat) + "," + str(lng) + ", '" + where + "']"
                fhand.write(output)
                
            except:
                continue

        fhand.write("\n];\n")
        
        self._cur.close()
        
        fhand.close()
        
        print("\n", count, " records written to where.js")
        print("\nOpen where.html to view the data in a browser")

In [12]:
importDada = geoload()

In [13]:
importDada.makeURL()

importDada.interactAPI()



...Retrieving... http://py4e-data.dr-chuck.net/json?address=AGH+University+of+Science+and+Technology&key=42

Retrieved  2324  characters {    "results" : [  


...Retrieving... http://py4e-data.dr-chuck.net/json?address=Academy+of+Fine+Arts+Warsaw+Poland&key=42

Retrieved  2039  characters {    "results" : [  


...Retrieving... http://py4e-data.dr-chuck.net/json?address=American+University+in+Cairo&key=42

Retrieved  1848  characters {    "results" : [  


...Retrieving... http://py4e-data.dr-chuck.net/json?address=Arizona+State+University&key=42

Retrieved  1789  characters {    "results" : [  


...Retrieving... http://py4e-data.dr-chuck.net/json?address=Athens+Information+Technology&key=42

Retrieved  1995  characters {    "results" : [  


...Retrieving... http://py4e-data.dr-chuck.net/json?address=BITS+Pilani&key=42

Retrieved  1822  characters {    "results" : [  


...Retrieving... http://py4e-data.dr-chuck.net/json?address=Babcock+University&key=42

Retrieved  1843  characte

In [16]:
writeLocations = geodump()

In [17]:
writeLocations.writeResults()

aleja Adama Mickiewicza 30, 30-059 Kraków, Poland 50.06688579999999 19.9136192
Krakowskie Przedmieście 5, 00-068 Warszawa, Poland 52.2394019 21.0150792
AUC Avenue، Road، New Cairo 1, Cairo Governorate 11835, Egypt 30.0186205 31.5014854
Tempe, AZ 85281, USA 33.4242399 -111.9280527
Monumental Plaza, Building C, 1st Floor, Leof. Kifisias 44, Marousi 151 25, Greece 38.0399391 23.8030901
Vidya Vihar, Pilani, Rajasthan 333031, India 28.3588163 75.58802039999999
121103, Ilishan-Remo, Nigeria 6.891986300000001 3.7181286
Ajagara, Varanasi, Uttar Pradesh 221005, India 25.2677203 82.99125819999999
Mysore Rd, Jnana Bharathi, Bengaluru, Karnataka 560056, India 12.9504048 77.5020617
1311 S 5th St, Waco, TX 76706, USA 31.5462231 -97.118059
19 Xin Wai Da Jie, Beitaipingzhuang, Hai Dian Qu, Bei Jing Shi, China, 100875 39.9619537 116.3662615
prasp. Niezaliežnasci 4, Minsk, Belarus 53.8930049 27.545623
Studentski trg 1, Beograd, Serbia 44.8184518 20.4575913
700 College St, Beloit, WI 53511, USA 42.503033

<hr>